## Perfilamiento con Kale:

In [1]:
!pip uninstall mex -y

Found existing installation: mex 1.0
Uninstalling mex-1.0:
  Successfully uninstalled mex-1.0


In [2]:
!pip install --quiet "git+https://github.com/lecepe00/mex_simplex.git#egg=mex&subdirectory=src"

In [3]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

## Características de la instancia

Utilizamos una instancia `m5.2xlarge` que entra en el programa de AWS Educate y que hemos ocupado en prácticas anteriores. Esta instancia tiene las siguientes características:

In [5]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:            7
CPU MHz:             3100.036
BogoMIPS:            4999.96
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand h

### Cargamos los datos:

In [19]:
mat = sio.loadmat('../data/AGG.mat')

In [20]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
#A.shape

# Prueba de paquetería `mex`

In [8]:
A = A.toarray()

In [9]:
minim = Minimizer(A, b, c)

In [10]:
minim.add_constraints(lbounds,ubounds)

In [16]:
#%timeit -n 1 -r 1 minim.solve()
minim.solve()

#### Valores aproximados

In [14]:
min_approx = minim.get_min()
coeff_approx = minim.get_coeff()
min_approx

-3.602879999999999e+35

## Prueba compilación a C

In [21]:
#mex_c
from mex.mex_c import general_c as mex_c_g
from mex.mex_c import simplex_networks_c as mex_c_sn
from mex.mex_c import problem_definition_c as mex_c_pd
from mex.mex_c.minimizer_class_c import Minimizer_c
from mex.mex_c.maximizer_class_c import Maximizer_c

In [ ]:
minim_c = Minimizer_c(A,b,c)

In [ ]:
minim_c.add_constraints(lbounds,ubounds)

In [ ]:
minim_c.matrix.shape

In [ ]:
%timeit -n 1 -r 1 minim_c.solve()

### Valores aproximados:

In [ ]:
min_approx_c = minim_c.get_min()
coeff_approx_c = minim_c.get_coeff()

In [ ]:
min_approx_c